In [1]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00


In [2]:
pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.5 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers faiss-cpu rank-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.0 MB/s eta 0:00:00


In [4]:
pip install streamlit -q

In [5]:
!wget -q -O - ipv4.icanhazip.com

34.145.253.221


In [7]:
%%writefile app.py

import json
import re
import nltk
from nltk.tokenize import sent_tokenize
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from rank_bm25 import BM25Okapi  # BM25 for hybrid search
import logging
import streamlit as st

# Download NLTK data
nltk.download('punkt', quiet=True)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class Hogragger:
    def __init__(self, corpus_path, model_name='sentence-transformers/all-MiniLM-L12-v2', qa_model='deepset/roberta-large-squad2', classifier_model='deepset/roberta-large-squad2'):
        self.corpus = self.load_corpus(corpus_path)
        self.cleaned_passages = self.preprocess_corpus()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        logging.info(f"Using device: {self.device}")

        # Initialize embedding model and build FAISS index
        self.model = SentenceTransformer(model_name).to(self.device)
        self.index = self.build_faiss_index()

        # Initialize BM25 for lexical matching
        self.bm25 = self.build_bm25_index()

        # Initialize classifier for question type prediction
        self.tokenizer = AutoTokenizer.from_pretrained(classifier_model)
        self.classifier = AutoModelForSequenceClassification.from_pretrained(classifier_model).to(self.device)

        # QA Model
        self.qa_model = pipeline('question-answering', model=qa_model, device=0 if self.device == 'cuda' else -1)

    def load_corpus(self, path):
        logging.info(f"Loading corpus from {path}")
        with open(path, "r") as f:
            corpus = json.load(f)
        logging.info(f"Loaded {len(corpus)} documents")
        return corpus

    def preprocess_corpus(self):
        cleaned_passages = []
        for article in self.corpus:
            body = article.get('body', '')
            clean_body = re.sub(r'<.*?>', '', body)  # Clean HTML tags
            clean_body = re.sub(r'\s+', ' ', clean_body).strip()  # Clean extra spaces
            cleaned_passages.append(self.create_passage(article, clean_body))
        logging.info(f"Created {len(cleaned_passages)} passages")
        return cleaned_passages

    def create_passage(self, article, chunk):
        return {
            "title": article['title'],
            "author": article.get('author', 'Unknown'),
            "published_at": article['published_at'],
            "category": article['category'],
            "url": article['url'],
            "source": article['source'],
            "passage": chunk.strip()
        }

    def build_faiss_index(self):
        logging.info("Building FAISS index...")
        embeddings = self.model.encode([p['passage'] for p in self.cleaned_passages], convert_to_tensor=True, device=self.device)
        embeddings = np.array(embeddings.cpu()).astype('float32')
        logging.info(f"Shape of embeddings: {embeddings.shape}")

        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        logging.info("Successfully created FAISS index")
        return index

    def build_bm25_index(self):
        logging.info("Building BM25 index...")
        tokenized_corpus = [p['passage'].split() for p in self.cleaned_passages]
        bm25 = BM25Okapi(tokenized_corpus)
        logging.info("Successfully built BM25 index")
        return bm25

    def predict_question_type(self, query):
        inputs = self.tokenizer(query, return_tensors='pt').to(self.device)
        outputs = self.classifier(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        labels = {0: 'inference_query', 1: 'comparison_query', 2: 'null_query', 3: 'temporal_query', 4: 'fact_query'}
        return labels.get(prediction, 'unknown_query')

    def retrieve_passages(self, query, k=100, threshold=0.7):
        query_embedding = self.model.encode([query], convert_to_tensor=True, device=self.device)
        D, I = self.index.search(np.array(query_embedding.cpu()), k)
        tokenized_query = query.split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        hybrid_scores = self.combine_faiss_bm25_scores(D[0], bm25_scores, I)
        passages = [self.cleaned_passages[i] for i, score in zip(I[0], hybrid_scores) if score > threshold]
        logging.info(f"Retrieved {len(passages)} passages using hybrid search for query.")
        return passages

    def combine_faiss_bm25_scores(self, faiss_scores, bm25_scores, passage_indices):
        bm25_scores = np.array(bm25_scores)[passage_indices]
        faiss_scores = np.array(faiss_scores)
        faiss_similarities = 1 / (faiss_scores + 1e-6)
        bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + 1e-6)
        faiss_similarities = (faiss_similarities - np.min(faiss_similarities)) / (np.max(faiss_similarities) - np.min(faiss_similarities) + 1e-6)
        combined_scores = 0.7 * faiss_similarities + 0.3 * bm25_scores
        return np.squeeze(combined_scores)

    def filter_passages(self, query, passages):
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        passage_embeddings = self.model.encode([p['passage'] for p in passages], convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(query_embedding, passage_embeddings)
        top_k = min(10, len(passages))
        top_indices = similarities.topk(k=top_k)[1].tolist()[0]
        selected_passages = []
        used_titles = set()
        for i in top_indices:
            if passages[i]['title'] not in used_titles:
                selected_passages.append(passages[i])
                used_titles.add(passages[i]['title'])
        return selected_passages

    def generate_answer(self, query, passages):
        context = " ".join([p['passage'] for p in passages[:5]])
        answer = self.qa_model(question=query, context=context)
        logging.info(f"Generated answer: {answer['answer']}")
        return answer['answer']

    def post_process_answer(self, answer, confidence=0.2):
        answer = re.sub(r'^.*\?', '', answer).strip()
        answer = answer.capitalize()
        if len(answer) > 100:
            truncated = re.match(r'^(.*?[.!?])\s', answer)
            if truncated:
                answer = truncated.group(1)
        if confidence < 0.2:
            logging.warning(f"Answer confidence too low: {confidence}")
            return "I'm unsure about this answer."
        return answer

    def process_query(self, query):
        question_type = self.predict_question_type(query)
        retrieved_passages = self.retrieve_passages(query, k=100, threshold=0.7)
        if not retrieved_passages:
            return {"query": query, "answer": "No relevant information found", "question_type": question_type, "evidence_list": []}

        filtered_passages = self.filter_passages(query, retrieved_passages)
        raw_answer = self.generate_answer(query, filtered_passages)

        evidence_count = min(len(filtered_passages), 4)
        evidence_list = [
            {
                "title": p['title'],
                "author": p['author'],
                "url": p['url'],
                "source": p['source'],
                "category": p['category'],
                "published_at": p['published_at'],
                "fact": self.extract_fact(p['passage'], query)
            } for p in filtered_passages[:evidence_count]
        ]
        final_answer = self.post_process_answer(raw_answer)

        return {
            "query": query,
            "answer": final_answer,
            "question_type": question_type,
            "evidence_list": evidence_list
        }

    def extract_fact(self, passage, query):
        sentences = sent_tokenize(passage)
        query_keywords = set(query.lower().split())
        best_sentence = max(sentences, key=lambda s: len(set(s.lower().split()) & query_keywords), default="")
        return best_sentence if best_sentence else (sentences[0] if sentences else "")

# Streamlit app
def main():
    hogragger = Hogragger(corpus_path='corpus.json')
    st.title("Hogragger Query Processor")
    query_input = st.text_area("Enter your query:", height=100)

    if st.button("Process Query"):
        if query_input:
            st.write(f"Processing query: {query_input}")
            result = hogragger.process_query(query_input)

            result_dict = {
                "query": query_input,
                "answer": result['answer'],
                "question_type": result['question_type'],
                "evidence_list": [
                    {
                        "title": ev['title'],
                        "fact": ev['fact'],
                        "source": ev['source'],
                        "url": ev['url'],
                        "published_at": ev['published_at'],
                        "category": ev['category']
                    }
                    for ev in result['evidence_list']
                ]
            }

            result_json = json.dumps(result_dict, indent=4)
            st.subheader("Result (JSON format):")
            st.code(result_json, language='json')
        else:
            st.error("Please enter a query.")

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.253.221:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://happy-eyes-unite.loca.lt
y
2024-10-14 17:08:32.075050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 17:08:32.110099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 17:08:32.119236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 17:08:32.141429: I tensorflow/core/platform/cpu_

In [6]:
from google.colab import files

uploaded = files.upload()

Saving corpus.json to corpus.json
